# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [13]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [14]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ta`u,-14.2336,-169.5144,29.20,79,100,8.38,AS,1712092181
1,1,port-aux-francais,-49.3500,70.2167,7.14,77,82,13.11,TF,1712092181
2,2,progreso,21.2833,-89.6667,40.06,43,45,3.77,MX,1712092181
3,3,yellowknife,62.4560,-114.3525,0.79,73,77,4.47,CA,1712092181
4,4,olonkinbyen,70.9221,-8.7187,-4.26,63,71,5.65,SJ,1712092182


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display

# Configure the map plot
all_city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM", 
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha = 0.5, 
    color = "Country"
)

# Display the map
all_city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Country,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values
weather_criteria_df = city_data_df.loc[(city_data_df["Max Temp"] < 50)
& (city_data_df["Max Temp"] > 20)
& (city_data_df["Cloudiness"] > 23)
& (city_data_df["Wind Speed"] < 3.5)                                       
& (city_data_df["Humidity"] < 30)
]

# Drop any rows with null values
weather_criteria_df.dropna()

# Display sample data
weather_criteria_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
178,178,fada n'gourma,12.0622,0.3578,35.94,11,100,3.30,BF,1712092206
194,194,shahpura,27.3833,75.9667,22.29,19,31,2.02,IN,1712092209
240,240,moundou,8.5667,16.0833,33.84,19,71,1.02,TD,1712092216
285,285,nguigmi,14.2495,13.1092,29.62,19,99,0.62,NE,1712092224
294,294,turbat,26.0023,63.0440,26.94,28,66,2.36,PK,1712092225
310,310,mandal,25.4500,74.5667,25.01,22,29,2.38,IN,1712092227
349,349,goure,13.9835,10.2704,30.94,9,100,3.42,NE,1712092233
378,378,mankera,31.3866,71.4407,23.33,24,70,3.44,PK,1712092237
407,407,zongolica,18.6667,-96.9833,35.79,22,85,3.48,MX,1712092240
445,445,bardai,11.6500,7.2333,30.05,14,27,3.46,NG,1712092246


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=weather_criteria_df.loc[:,["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
178,fada n'gourma,BF,12.0622,0.3578,11,
194,shahpura,IN,27.3833,75.9667,19,
240,moundou,TD,8.5667,16.0833,19,
285,nguigmi,NE,14.2495,13.1092,19,
294,turbat,PK,26.0023,63.0440,28,
310,mandal,IN,25.4500,74.5667,22,
349,goure,NE,13.9835,10.2704,9,
378,mankera,PK,31.3866,71.4407,24,
407,zongolica,MX,18.6667,-96.9833,22,
445,bardai,NG,11.6500,7.2333,14,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [25]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accomodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=row["Lat"]
    longitude=row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{latitude},{longitude},{radius}"
    params["bias"] = f"proximity:{latitude},{longitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
fada n'gourma - nearest hotel: No hotel found
shahpura - nearest hotel: No hotel found
moundou - nearest hotel: No hotel found
nguigmi - nearest hotel: No hotel found
turbat - nearest hotel: No hotel found
mandal - nearest hotel: No hotel found
goure - nearest hotel: No hotel found
mankera - nearest hotel: No hotel found
zongolica - nearest hotel: No hotel found
bardai - nearest hotel: No hotel found
illizi - nearest hotel: No hotel found
moriki - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
178,fada n'gourma,BF,12.0622,0.3578,11,No hotel found
194,shahpura,IN,27.3833,75.9667,19,No hotel found
240,moundou,TD,8.5667,16.0833,19,No hotel found
285,nguigmi,NE,14.2495,13.1092,19,No hotel found
294,turbat,PK,26.0023,63.0440,28,No hotel found
310,mandal,IN,25.4500,74.5667,22,No hotel found
349,goure,NE,13.9835,10.2704,9,No hotel found
378,mankera,PK,31.3866,71.4407,24,No hotel found
407,zongolica,MX,18.6667,-96.9833,22,No hotel found
445,bardai,NG,11.6500,7.2333,14,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [28]:
%%capture --no-display

# Configure the map plot
configure_hotel =hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="OSM",
    frame_width=500,
    frame_height=300,
    size="Humidity",
    alpha=0.8,
    color="City",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
configure_hotel

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)